# Grouping Data with SQL

## Introduction

Just as with Pandas, another very useful manipulation in SQL is aggregate functions. For example, you may wish to find the mean, median, min or max of a column feature. For example, in the customer relational database that you've been working with, you may wonder if there are differences in overall sales across offices or regions.

## Objectives

You will be able to:

* Write queries with aggregate functions like `COUNT`, `MAX`, `MIN`, and `SUM`
* Create an alias for the return value of an aggregate function
* Use `GROUP BY` to sort the data sets returned by aggregate functions
* Compare aggregates using the `HAVING` clause

## Database Schema
<img src="images/Database-Schema.png">

In [1]:
import sqlite3
import pandas as pd

## Connecting to the Database

As usual, start by creating a connection to the database.

In [2]:
conn = sqlite3.Connection('data.sqlite')
c = conn.cursor()
cur = conn.cursor()

In [3]:
df = pd.DataFrame(
    c.execute("""
    SELECT *
    FROM sqlite_master 
    WHERE type='table';
""").fetchall()
)
df.columns = [x[0] for x in c.description]
c.description
df

,type,name,tbl_name,rootpage,sql
0,table,orderdetails,orderdetails,2,"CREATE TABLE `orderdetails` (`orderNumber`, `p..."
1,table,payments,payments,28,"CREATE TABLE `payments` (`customerNumber`, `ch..."
2,table,offices,offices,32,"CREATE TABLE `offices` (`officeCode`, `city`, ..."
3,table,customers,customers,33,"CREATE TABLE `customers` (`customerNumber`, `c..."
4,table,orders,orders,38,"CREATE TABLE `orders` (`orderNumber`, `orderDa..."
5,table,productlines,productlines,46,"CREATE TABLE `productlines` (`productLine`, `t..."
6,table,products,products,47,"CREATE TABLE `products` (`productCode`, `produ..."
7,table,employees,employees,56,"CREATE TABLE `employees` (`employeeNumber`, `l..."


## Groupby and Aggregate Functions

Lets start by looking at some groupby statements to aggregate our data.

In [4]:
df = pd.DataFrame(
    c.execute("""
SELECT *
FROM offices
""").fetchall()
)
df.columns = [x[0] for x in c.description]
c.description
df


,officeCode,city,phone,addressLine1,addressLine2,state,country,postalCode,territory
0,1,San Francisco,+1 650 219 4782,100 Market Street,Suite 300,CA,USA,94080,NA
1,2,Boston,+1 215 837 0825,1550 Court Place,Suite 102,MA,USA,02107,NA
2,3,NYC,+1 212 555 3000,523 East 53rd Street,apt. 5A,NY,USA,10022,NA
3,4,Paris,+33 14 723 4404,43 Rue Jouffroy D'abbans,,,France,75017,EMEA
4,5,Tokyo,+81 33 224 5000,4-1 Kioicho,,Chiyoda-Ku,Japan,102-8578,Japan
5,6,Sydney,+61 2 9264 2451,5-11 Wentworth Avenue,Floor #2,,Australia,NSW 2010,APAC
6,7,London,+44 20 7877 2041,25 Old Broad Street,Level 7,,UK,EC2N 1HN,EMEA


In [5]:
#Here we join the offices and employees tables in order to count the number of employees per city.
cur.execute("""select city,
                      count(employeeNumber)
                      from offices
                      join employees
                      using(officeCode)
                      group by city
                      order by count(employeeNumber) desc;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,count(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## Aliasing
You can also alias our groupby by specifying the number of our selection order that we want to group by. This is simply written as `group by 1` 1 referring to the first column name that we are selecting.

Additionally, we can also rename our aggregate to a more descriptive name using the `as` clause.

In [6]:
cur.execute("""select city,
                      count(employeeNumber) as numEmployees
                      from offices
                      join employees
                      using(officeCode)
                        group by 1

                      order by numEmployees desc;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Boston,2
4,London,2


## Other Aggregations

Aside from count() some other useful aggregations include:
    * min()
    * max()
    * sum()
    * avg()

In [8]:
cur.execute("""select customerName,
                      count(*) as number_purchases,
                      min(amount) as min_purchase,
                      max(amount) as max_purchase,
                      avg(amount) as avg_purchase,
                      sum(amount) as total_spent
                      from customers
                      join payments
                      using(customerNumber)
                      group by 1
                      order by sum(amount) desc;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

98


,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03


In [9]:
df.tail()

,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
93,Royale Belge,4,1128.20,1627.56,7304.295000,29217.18
94,Frau da Collezione,2,17746.26,7612.06,12679.160000,25358.32
95,Atelier graphique,3,14571.44,6066.78,7438.120000,22314.36
96,Auto-Moto Classics Inc.,3,5858.56,9658.74,7184.753333,21554.26
97,Boards & Toys Co.,2,3452.75,4465.85,3959.300000,7918.60


## The having clause

Finally, we can also filter our aggregated views with the having clause. The having clause works like the where clause but is used to filter data selections on conditions post the group by. For example, if we wanted to filter based on a customer's last name, we would use the where clause. However, if we wanted to filter a list of city's with at least 5 customers, we would using the having clause; we would first groupby city and count the number of customers, and the having clause allows us to pass conditions on the result of this aggregation.

In [10]:
cur.execute("""select city,
                      count(customerNumber) as number_customers
                      from customers
                      group by 1
                      ;
""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

96


,city,number_customers
0,Aachen,1
1,Allentown,1
2,Amsterdam,1
3,Auckland,1
4,Auckland,2


## Combining the where and having clause
We can also use the where and having clause in conjunction with each other for more complex rules.
For example, let's say we want a list of customers who have made at least 3 purchases of over 50K each.

In [11]:
cur.execute("""select customerName,
                      count(amount) as number_purchases_over_50K
                      from customers
                      join payments
                      using(customerNumber)
                      where amount >= 50000
                      group by 1
                      having count(amount) >= 3
                      order by count(amount) desc;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

53


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Anna's Decorations, Ltd",4
3,"Australian Collectors, Co.",4
4,Baane Mini Imports,4


In [12]:
df.tail()

,customerName,number_purchases_over_50K
48,"Stylish Desk Decors, Co.",3
49,Suominen Souveniers,3
50,"Toys of Finland, Co.",3
51,Toys4GrownUps.com,3
52,Vitachrome Inc.,3


## Summary

After this section, you should have a good idea of how to use aggregate functions, aliases and the having clause to filter selections.